In [13]:
!pip install langchain langchain-groq

In [ ]:
import os
import re
import time
from typing import List, Tuple, Optional
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from tqdm import tqdm


class SimilarityResult(BaseModel):
    """Output schema for semantic similarity analysis."""
    entity_match_score: float = Field(..., description="Score for how well entities match between texts (0-1)")
    concept_match_score: float = Field(..., description="Score for how well main concepts match (0-1)")
    context_match_score: float = Field(..., description="Score for how well context and details match (0-1)")
    perspective_match_score: float = Field(..., description="Score for how well tone and perspective match (0-1)")
    final_similarity_score: float = Field(..., description="Overall semantic similarity score (0-5)")
    reasoning: str = Field(..., description="Detailed analysis explaining the similarity assessment")


class SemanticSimilarityAnalyzer:
    """A class for analyzing semantic similarity between texts using LLM reasoning."""
    
    def __init__(
        self, 
        model_name: str = "llama3-8b-8192",  
        api_key: Optional[str] = "api_key",
        temperature: float = 0.1,  # Low temperature for more consistent reasoning
        max_retries: int = 3,
        retry_delay: int = 2
    ):
        """Initialize the semantic similarity analyzer.
        
        Args:
            model_name: Name of the Groq model to use
            api_key: Groq API key (if None, tries to get from environment)
            temperature: Temperature setting for generation
            max_retries: Maximum number of retries for API calls
            retry_delay: Delay in seconds between retries
        """
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        if not self.api_key:
            raise ValueError("Groq API key must be provided or set as GROQ_API_KEY environment variable")
        
        self.llm = ChatGroq(
            model_name=model_name,
            api_key=self.api_key,
            temperature=temperature
        )
        self.max_retries = max_retries
        self.retry_delay = retry_delay
        self.output_parser = PydanticOutputParser(pydantic_object=SimilarityResult)
        
    def _create_prompt(self, text1: str, text2: str) -> str:
        """Create a detailed prompt for semantic similarity analysis."""
        prompt = f"""You are an expert in semantic analysis with deep understanding of language nuance.

Task: Analyze the semantic similarity between two texts thoroughly, then provide structured scoring.

Text A: "{text1}"

Text B: "{text2}"

Approach your analysis methodically:

1. Entity Analysis: Identify key entities (people, objects, concepts) in both texts. Are they the same? Similar? Different?

2. Concept Mapping: Extract and compare the main ideas, themes, and purposes of both texts.

3. Context Evaluation: Analyze details, examples, time references, locations, and supporting information.

4. Perspective Assessment: Compare tone, viewpoint, implied stance, and emotional content.

Through your analysis, assign scores for each dimension (0.0-1.0) and provide a final similarity score (0.0-5.0).

{self.output_parser.get_format_instructions()}

Remember that genuinely similar texts should convey essentially the same core meaning, even if they use different words or structures.
"""
        return prompt
    
    def analyze_similarity(self, text1: str, text2: str) -> SimilarityResult:
        """Analyze the semantic similarity between two texts.
        
        Args:
            text1: First text for comparison
            text2: Second text for comparison
            
        Returns:
            SimilarityResult object containing scores and reasoning
        """
        prompt = self._create_prompt(text1, text2)
        
        # Implement retry logic for API stability
        for attempt in range(self.max_retries):
            try:
                response = self.llm.invoke([HumanMessage(content=prompt)])
                # Extract the JSON part from the response
                match = re.search(r'\{[\s\S]*\}', response.content)
                if match:
                    json_str = match.group(0)
                    return self.output_parser.parse(json_str)
                else:
                    # If no JSON found, try to handle the raw response
                    return self._parse_unstructured_response(response.content)
            except Exception as e:
                if attempt < self.max_retries - 1:
                    print(f"Attempt {attempt+1} failed with error: {str(e)}. Retrying in {self.retry_delay}s...")
                    time.sleep(self.retry_delay)
                else:
                    raise Exception(f"Failed to analyze similarity after {self.max_retries} attempts: {str(e)}")
    
    def _parse_unstructured_response(self, content: str) -> SimilarityResult:
        """Attempt to parse an unstructured response into our schema."""
        # Extract the final score
        score_match = re.search(r'(?:final|overall)?\s*(?:similarity)?\s*score:?\s*(\d+\.?\d*)', 
                              content.lower(), re.IGNORECASE)
        final_score = float(score_match.group(1)) if score_match else 2.5  # Default mid-point
        
        # Default scores when structured parsing fails
        return SimilarityResult(
            entity_match_score=final_score/5.0,
            concept_match_score=final_score/5.0,
            context_match_score=final_score/5.0,
            perspective_match_score=final_score/5.0,
            final_similarity_score=final_score,
            reasoning=content
        )
    
    def batch_analyze(self, text_pairs: List[Tuple[str, str]], batch_size: int = 5) -> List[SimilarityResult]:
        """Process multiple text pairs in batches.
        
        Args:
            text_pairs: List of (text1, text2) tuples to compare
            batch_size: Number of pairs to process before reporting progress
            
        Returns:
            List of SimilarityResult objects
        """
        results = []
        
        for i in tqdm(range(0, len(text_pairs), batch_size), desc="Processing batches"):
            batch = text_pairs[i:i+batch_size]
            batch_results = []
            
            for text1, text2 in batch:
                try:
                    result = self.analyze_similarity(text1, text2)
                    batch_results.append(result)
                except Exception as e:
                    print(f"Error processing pair: {str(e)}")
                    # Add a placeholder result
                    batch_results.append(SimilarityResult(
                        entity_match_score=0.0,
                        concept_match_score=0.0,
                        context_match_score=0.0,
                        perspective_match_score=0.0,
                        final_similarity_score=0.0,
                        reasoning=f"Error: {str(e)}"
                    ))
            
            results.extend(batch_results)
            
        return results

In [14]:
para1 = """The implementation of artificial intelligence in healthcare systems has shown promising results 
in diagnostics and treatment planning. Recent studies indicate that AI algorithms can detect 
certain conditions with accuracy comparable to that of experienced physicians, while potentially 
reducing diagnostic time. However, concerns about data privacy and the black-box nature of some 
AI models remain significant obstacles to widespread adoption."""

para2 = """Healthcare systems have benefited from artificial intelligence applications, particularly 
in the areas of diagnosis and treatment recommendations. Research demonstrates that machine learning 
models can identify some medical conditions as accurately as seasoned doctors, often in less time. 
Nevertheless, issues related to patient privacy and the interpretability of complex AI systems continue 
to present challenges for broader implementation."""


analyzer = SemanticSimilarityAnalyzer()


result = analyzer.analyze_similarity(para1, para2)


print(f"Final Similarity Score: {result.final_similarity_score}/5.0")
print(f"Entity Match: {result.entity_match_score:.2f}/1.0")
print(f"Concept Match: {result.concept_match_score:.2f}/1.0")
print(f"Context Match: {result.context_match_score:.2f}/1.0")
print(f"Perspective Match: {result.perspective_match_score:.2f}/1.0")
print(f"Reasoning: {result.reasoning}")


Final Similarity Score: 4.5/5.0
Entity Match: 0.80/1.0
Concept Match: 0.90/1.0
Context Match: 0.80/1.0
Perspective Match: 1.00/1.0
Reasoning: The two texts share a high degree of semantic similarity, with most entities, concepts, and context being identical or very similar. The main ideas and sub-ideas are also very similar, with some minor differences in wording. The tone, viewpoint, and emotional content are identical. The only minor differences are in the specific examples and wording used. Overall, the texts convey essentially the same core meaning, making them highly similar.
